### Remove mixed pixels

In [ ]:
osmlulc = '/home/gisuser/autocls/osmtolc_res.gpkg'

osmlyr = 'osmlc_lisbon_harm'

lulc_col = 'idclcp'

refimg = '/home/gisuser/autocls/T29SMC_20180818T112111_B02_10m.tif'

out = '/home/gisuser/autocls/osmlc_lisbon_nomixpx.tif'

In [ ]:
import os
import datetime as dt
from glass.dtt.rst.toshp import rst_to_polyg
from glass.gp.ovl.grs    import grsunion
from glass.pys.oss import fprop
from glass.wenv.grs import run_grass
from glass.pys.tm import now_as_str
from glass.it.shp import shp_to_grs, grs_to_shp
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.rst.rcls import onlydata_rules
from glass.rst.rcls import rcls_rst
from glass.dtt.split import split_shp_by_attr
from glass.dtt.cg.grs import v_to_lines
from glass.sql.db import create_pgdb
from glass.it.db import shp_to_psql
from glass.sql.q import q_to_ntbl
from glass.it.shp import dbtbl_to_shp
from glass.dtt.cg.sql import polyg_to_lines
from glass.prop.prj import get_epsg, shp_epsg
from glass.dtt.torst import grsshp_to_grsrst
from glass.rst.rcls import null_to_value, set_null
from glass.dtt.torst import shp_to_rst
from glass.prj import proj
from glass.rst.alg import grsrstcalc
from glass.rst.mos import rsts_to_mosaic

In [ ]:
# Setup workspace and GRASS GIS Session
# Check if outfolder exists
ws = os.path.dirname(out)
if not os.path.exists(ws):
    mkdir(ws, overwrite=None)

"""
Start GRASS GIS Session
"""

loc = f'loc_{now_as_str()}'
grsb = run_grass(
    ws, grassBIN='grass78', location=loc,
    srs=refimg
)

import grass.script.setup as gsetup
    
gsetup.init(grsb, ws, loc, 'PERMANENT')

In [ ]:
# Temp Geopackage
tmpgpkg = os.path.join(ws, loc, 'tmpdata.gpkg')

In [ ]:
iepsg = shp_epsg(osmlulc, lyrname=osmlyr)
oepsg = get_epsg(refimg)

# Re-project data if necessary
if iepsg != oepsg:
    osmlulc = proj(
        osmlulc, tmpgpkg, oepsg, inEPSG=iepsg,
        api="ogr2ogr", ilyr=osmlyr, olyr=osmlyr
    )

In [ ]:
# Create DB
time_a = dt.datetime.now().replace(microsecond=0)
db = create_pgdb(fprop(out, 'fn'), overwrite=True)

In [ ]:
# Send data to the database
geotbl = shp_to_psql(db, osmlulc, api='ogr2ogr', lyrname={osmlulc: osmlyr})

# Transform polygons into lines
linelyr = 'lulc_lines'
linetbl = polyg_to_lines(
    db, geotbl, 'geom', tmpgpkg,
    out_is_file=True, olyr='lulc_lines'
)

In [ ]:
# Add data to GRASS GIS
grslnh = shp_to_grs(linetbl, lyrname=linelyr, asCMD=True)
grsimg = rst_to_grs(refimg, as_cmd=True)

time_b = dt.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
# LULC Classes to Raster

# First, each class to a new layer
lclyrs = split_shp_by_attr(
    osmlulc, lulc_col,
    tmpgpkg, ilyr=osmlyr, outname='lulc',
    valinname=True
)

# Second convert each layer to raster
for shp in lclyrs:
    lclyrs[shp] = shp_to_rst(
        tmpgpkg, 1, 10, 0,
        os.path.join(ws, loc, f'rst_lulc_{shp}.tif'),
        rst_template=refimg, lyrname=lclyrs[shp],
        api='gdal', rtype=int, dtype='Byte'
    )

In [ ]:
lclyrs

In [ ]:
# Lines to Raster
lnhrst = grsshp_to_grsrst(grslnh, 1, 'rstlines', cmd=True)

null_to_value(lnhrst, 0, as_cmd=True)
set_null(lnhrst, 1, ascmd=True)

In [ ]:
# For each class
# Add it to GRASS GIS
# Remove Mixed cells

# Reclassify: class present = 1; ausence=0
for cls in lclyrs:
    lclyrs[cls] = rst_to_grs(lclyrs[cls], as_cmd=True)

    lclyrs[cls] = grsrstcalc(f"{lclyrs[cls]} + {lnhrst}", f"{lclyrs[cls]}_v1")

    null_to_value(lclyrs[cls], 0, as_cmd=True)

# Sum all class rasters to know were we have
# cells with more than one class
sumrst = grsrstcalc(" + ".join(list(lclyrs.values())), 'overlaycells')
fcells = grsrstcalc(f"if({sumrst} > 1, null(), 0)", 'cellsmantain')

# For each class, remove cells with more than one class
for cls in lclyrs:
    set_null(lclyrs[cls], 0, ascmd=True)
    lclyrs[cls] = grsrstcalc(
        f"{lclyrs[cls]} + {fcells} + {cls} - 1",
        f"{lclyrs[cls]}_v2"
    )

# Create final raster
frst = rsts_to_mosaic(list(lclyrs.values()), fprop(out, 'fn'))

In [ ]:
# Export final result
grs_to_rst(frst, out, as_cmd=False, rtype=int, dtype='UInt16', nodata=0)